# DCGAN - Supplemental Media and Figures

Contains supplemental media and figures, currently includes the following:

- Training Progress Video 
- Training Progress Gif
- Training Loss
- CPU Generated Samples

--------

In [ ]:
# General Deps
import os
import re
import datetime
import numpy as np

# Plotting + Video
import ffmpeg
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# DCGAN
import gaudi_dcgan as dcgan

In [ ]:
# Get Start Datetime for plots; apply all the way through the Notebook
PLOT_DTTM = re.sub(":|-| |\.", "_", datetime.datetime.utcnow().__str__())

In [ ]:
# Init Model and Training Configs w. Default Values - See gaudi_dcgan.py for descriptions. For clarity,
# objects below are initialized with their default values.
model_cfg = dcgan.ModelCheckpointConfig(
    batch_size=128,
    img_size=64,
    nc=3,
    nz=100,
    ngf=64,
    ndf=64,
    lr=0.0002,
    beta1=0.5,
    beta2=0.999,
)

train_cfg = dcgan.TrainingConfig(
    model_name="msls_dcgan_ml_p3_8xlarge_001", # Custom Model Name To Identify Gaudi vs GPU trained
    model_dir="/efs/trained_model",
    save_frequency=1,
    log_frequency=50,
    gen_progress_frequency=250,
)

In [ ]:
# Create Figures and Videos Directory if Not Yet Exists...
if not os.path.exists(f"{model_cfg.model_dir}/{model_cfg.model_name}/figures"):
    os.makedirs(f"{model_cfg.model_dir}/{model_cfg.model_name}/figures")

if not os.path.exists(f"{model_cfg.model_dir}/{model_cfg.model_name}/videos"):
    os.makedirs(f"{model_cfg.model_dir}/{model_cfg.model_name}/videos")

## Figure 1.1 - `G` and `D` Training Losses

In [ ]:
# Plot the Training Losses to the Generator (G) and Discriminator (D)

plt.figure(figsize=(10, 5))
plt.title(f"Generator and Discriminator Loss During Training - {model_cfg.model_name}")
plt.plot(result["losses"]["_G"], label="G")
plt.plot(result["losses"]["_D"], label="D")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.savefig(
    f"{model_cfg.model_dir}/{model_cfg.model_name}/figures/train_loss_{plots_execution_dttm}.png"
)

## Figure 2.1.1 - Final Images of Fixed Noise Sample

In [ ]:
# Plot a Graphic of the Final images...

fig = plt.figure(figsize=(8, 8))
plt.axis("off")

# Create a Frame for each epoch from results.img_list
ims = [
    [plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)] for i in result["img_list"]
]

# 2.1.1 Final Images of Fixed Noise Sample
plt.savefig(
    f"{model_cfg.model_dir}/{model_cfg.model_name}/figures/generaed_{plots_execution_dttm}.png"
)

## Figure 2.1.2 - Training Progress Sequence on Fixed Noise

In [ ]:
# 2.1.1 - Training Progress Sequence Saved as Video

ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

content = HTML(ani.to_jshtml())

writergif = animation.PillowWriter(
    fps=10, metadata=dict({"title": f"{model_cfg.model_name}"}, **model_cfg.__dict__)
)

In [ ]:
# Save Animation as Gif and as HTML w. Video...
ani.save(
    f"{model_cfg.model_dir}/{model_cfg.model_name}/videos/progress_{plots_execution_dttm}.gif",
    writer=writergif,
)

with open(
    f"{model_cfg.model_dir}/{model_cfg.model_name}/videos/progress_{plots_execution_dttm}.html",
    "w",
) as fi:
    print(ani.to_html5_video(), file=fi)

In [ ]:
# Grab a batch of real images from the dataloader and compare the final Generated images vs the Real
# images. Do they hold up against human discretion?

real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15, 15))

plt.subplot(1, 2, 1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(
    np.transpose(
        vutils.make_grid(
            real_batch[0].to(train_cfg.dev)[:64], padding=5, normalize=True
        ).cpu(),
        (1, 2, 0),
    )
)

In [ ]:
# Plot the fake images from the final epoch
# NOTE: This uses the fixed noise from the trained model and is not variable between executions

plt.subplot(1, 2, 2)
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(result["img_list"][-1], (1, 2, 0)))
plt.show()

plt.savefig(
    f"{model_cfg.model_dir}/{model_cfg.model_name}/figures/compare_{plots_execution_dttm}.png"
)

In [ ]:
# Generate a few sample images; this is using randomly generated noise
# and results should be variable across multiple runs...
# Experiment: Ideally a Generator Net can use a CPU to (slowly) generate samples, being able to move
# the G network off GPU/HPU allows us to serve imgs off an inexpensive webserver. In this case we generator
# noise AND images on the CPU

plt.figure(figsize=(15, 15))

# generated_data[0].shape == torch.Size([3, 64, 64])
train_cfg_cpu_only = dcgan.TrainingConfig(dev=torch.device("cpu"))

imgs = dcgan.generate_fake_samples(
    n_samples=4, train_cfg=train_cfg_cpu_only, model_cfg=model_cfg, as_of_epoch=4
)


plt.imshow(
    np.transpose(
        vutils.make_grid(imgs.to(train_cfg.dev), padding=2, normalize=True).cpu(),
        (1, 2, 0),
    )
)